In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import seaborn as sn
sn.set_context("poster")

import torch
from torch import nn as nn
ttype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
ctype = torch.cuda.LongTensor if torch.cuda.is_available() else torch.LongTensor
print(ttype)
import torch.nn.functional as F
from matplotlib import gridspec
from sithcon import SITHCon_Layer, _SITHCon_Core, iSITH

from tqdm.notebook import tqdm

import itertools
from csv import DictWriter
import os 
from os.path import join
import glob

import numpy as np
import pandas as pd
import pickle
from math import factorial
import random

<class 'torch.cuda.FloatTensor'>


In [3]:
full_path = join('/media', 'root', 'easystore', 'experiments', 
                 'audiomnist', '01.00', 'AudioMNIST', 'data')

In [4]:
all_files = pd.read_csv(join('data','files_info.csv'))
all_dat = []
for f in glob.glob(join(full_path, "*")):
    print(f)
    all_dat.append(np.load(join(full_path, f)))
all_dat = np.concatenate(all_dat, axis=0)

/media/root/easystore/experiments/audiomnist/01.00/AudioMNIST/data/data_17_1.00.npy
/media/root/easystore/experiments/audiomnist/01.00/AudioMNIST/data/data_34_1.00.npy
/media/root/easystore/experiments/audiomnist/01.00/AudioMNIST/data/data_01_1.00.npy
/media/root/easystore/experiments/audiomnist/01.00/AudioMNIST/data/data_02_1.00.npy
/media/root/easystore/experiments/audiomnist/01.00/AudioMNIST/data/data_03_1.00.npy
/media/root/easystore/experiments/audiomnist/01.00/AudioMNIST/data/data_04_1.00.npy
/media/root/easystore/experiments/audiomnist/01.00/AudioMNIST/data/data_05_1.00.npy
/media/root/easystore/experiments/audiomnist/01.00/AudioMNIST/data/data_06_1.00.npy
/media/root/easystore/experiments/audiomnist/01.00/AudioMNIST/data/data_07_1.00.npy
/media/root/easystore/experiments/audiomnist/01.00/AudioMNIST/data/data_08_1.00.npy
/media/root/easystore/experiments/audiomnist/01.00/AudioMNIST/data/data_09_1.00.npy
/media/root/easystore/experiments/audiomnist/01.00/AudioMNIST/data/data_10_1

In [5]:
all_files['ind_idx'] = all_files.dat_idx[:]
all_files.dat_idx =  (all_files.subjid - 1)*500 + all_files.dat_idx

In [7]:
trainX = all_dat[list(all_files.loc[(all_files.rec_split<9)&(all_files.subjid!=60), 'dat_idx'])]
trainY = np.array(all_files.loc[(all_files.rec_split<9)&(all_files.subjid!=60), 'digit'])
testX = all_dat[list(all_files.loc[(all_files.rec_split==9)&(all_files.subjid!=60), 'dat_idx'])]
testY = np.array(all_files.loc[(all_files.rec_split==9)&(all_files.subjid!=60), 'digit'])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

(26550, 50, 250) (26550,) (2950, 50, 250) (2950,)


In [9]:
batch_size = 16
dataset = torch.utils.data.TensorDataset(torch.Tensor(trainX).cuda(), 
                                         torch.LongTensor(trainY).cuda())
dataset = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

dataset_valid = torch.utils.data.TensorDataset(torch.Tensor(testX).cuda(), 
                                               torch.LongTensor(testY).cuda())
dataset_valid = torch.utils.data.DataLoader(dataset_valid, batch_size=batch_size, shuffle=False)

# Classes 

In [10]:
class SITHCon_Classifier(nn.Module):
    def __init__(self, out_classes, layer_params, 
                 act_func=nn.ReLU, batch_norm=False,
                 dropout=.2):
        super(SITHCon_Classifier, self).__init__()
        last_channels = layer_params[-1]['channels']
        self.transform_linears = nn.ModuleList([nn.Linear(l['channels'], l['channels'])
                                                for l in layer_params])
        self.sithcon_layers = nn.ModuleList([SITHCon_Layer(l, act_func) for l in layer_params])
        self.to_out = nn.Linear(last_channels, out_classes)
        
        
    def forward(self, inp):
        
        x = inp
        #out = []
        for i in range(len(self.sithcon_layers)):
            x = self.sithcon_layers[i](x)
            
            x = F.relu(self.transform_linears[i](x[:,0,:,:].transpose(1,2)))
            x = x.unsqueeze(1).transpose(2,3)

            #out.append(x.clone())
        x = x.transpose(2,3)[:, 0, :, :]
        #x = x.transpose(2,3)[:, 0, :, :]
        x = self.to_out(x)
        return x

# Functions

In [15]:
def gen_model(p):
    sp1 = dict(in_features=50, 
               tau_min=.1, tau_max=4000, buff_max=6500,
               dt=1, ntau=p[0], k=p[1], g=0.0, ttype=ttype, 
               channels=35, kernel_width=p[2], dilation=p[3],
               dropout=None, batch_norm=None)
    sp2 = dict(in_features=sp1['channels'], 
               tau_min=.1, tau_max=4000, buff_max=6500,
               dt=1, ntau=p[0], k=p[1], g=0.0, ttype=ttype, 
               channels=35, kernel_width=p[2], dilation=p[3], 
               dropout=None, batch_norm=None)
    sp3 = dict(in_features=sp2['channels'], 
               tau_min=.1, tau_max=4000, buff_max=6500,
               dt=1, ntau=p[0], k=p[1], g=0.0, ttype=ttype, 
               channels=35, kernel_width=p[2], dilation=p[3], 
               dropout=None, batch_norm=None)
    layer_params = [sp1, sp2]#, sp3]
    model = SITHCon_Classifier(10, layer_params, act_func=None,#nn.ReLU
                              ).cuda()
    return model

def train(model, ttype, train_loader, test_loader, optimizer, loss_func, epoch, perf_file,
          loss_buffer_size=100, batch_size=4, device='cuda',
          prog_bar=None, last_test_perf=0):
    
    assert(loss_buffer_size%batch_size==0)
    perfs = []
    losses = []
    best_test_perf = last_test_perf
    
    for batch_idx, (data, target) in enumerate(train_loader):
        model.train()
        data = data.to(device).unsqueeze(1)
        target = target.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = loss_func(out[:, -1, :],
                         target)

        loss.backward()
        optimizer.step()
        perfs.append((torch.argmax(out[:, -1, :], dim=-1) == 
                      target).sum().item())
        perfs = perfs[int(-loss_buffer_size/batch_size):]
                          
        losses.append(loss.detach().cpu().numpy())
        losses = losses[int(-loss_buffer_size/batch_size):]
        
        if ((batch_idx*batch_size)%loss_buffer_size == 0) & (batch_idx != 0):
            loss_track = {}
            #last_test_perf = test_model(model, 'cuda', test_loader, 
            #                            batch_size)
            loss_track['avg_loss'] = np.mean(losses)
            #loss_track['last_test'] = last_test_perf
            loss_track['training_perf'] = np.mean(perfs) / (batch_size)
            loss_track['epoch'] = epoch
            loss_track['batch_idx'] = batch_idx
            last_test_perf =  np.mean(perfs) / (batch_size)
            with open(perf_file, 'a+') as fp:
                csv_writer = DictWriter(fp, fieldnames=list(loss_track.keys()))
                if fp.tell() == 0:
                    csv_writer.writeheader()
                csv_writer.writerow(loss_track)
                fp.flush()
            if best_test_perf < last_test_perf:
                torch.save(model.state_dict(), perf_file[:-4]+".pt")
                best_test_perf = last_test_perf
        if not (prog_bar is None):
            # Update progress_bar
            s = "{}:{} Loss: {:.5f}, Tperf: {:.4f}, valid: {:.4f}"
            format_list = [epoch,batch_idx*batch_size, np.mean(losses), 
                           np.mean(perfs) / (batch_size),
                           last_test_perf]         
            s = s.format(*format_list)
            prog_bar.set_description(s)
    return best_test_perf
def test_model(model, device, test_loader, batch_size):
    # Test the Model
    perfs = []
    tot = 0.0
    total_num = 0.0
    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(device).unsqueeze(1)
            batch = data.shape[0]
            target = target.to(device)
            out = model(data)
            total_num += batch
            tot += (torch.argmax(out[:, -1, :], 
                                 dim=-1) == target).sum().item()
            
    perf = tot / total_num
    return perf

def save_outcome(outcome, filename):
    dat = pd.DataFrame(outcome)
    dat.to_csv(join('perf',filename))

# Go!

In [16]:
params = [
          [400, 35, 23, 2],
          ]
model = gen_model(params[0])

In [17]:
runs = 1
for r in range(runs):
    for i, p in enumerate(params):
        best_test_perf = -1
        model = gen_model(p)

        tot_weights = 0
        for p in model.parameters():
            tot_weights += p.numel()
        print("Total Weights:", tot_weights)

        loss_func = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters())
        epochs = 10
        Trainscale = 10
        device='cuda'
        progress_bar = tqdm(range(int(epochs)), bar_format='{l_bar}{bar:5}{r_bar}{bar:-5b}')
        epochperfs = []
        times_100 = 0
        for epoch_idx in progress_bar:
            perfs = []
            losses = []
            model.train()
            best_test_perf = train(model, ttype, dataset, dataset_valid, 
                                  optimizer, loss_func, batch_size=batch_size, loss_buffer_size=992,
                                  epoch=epoch_idx, perf_file=join('perf','sithcon_audiomnist_5032021_{}.csv'.format(r)),
                                  prog_bar=progress_bar, last_test_perf=best_test_perf)

Total Weights: 71375


In [18]:
model.load_state_dict(torch.load('perf/sithcon_audiomnist_5032021_0.pt'))

<All keys matched successfully>

In [19]:
scales = [
          '10.00', 
          '05.00', 
          '02.50',
          '01.25', 
          '01.00', 
          '00.80', 
          '00.40', 
          '00.20', 
          '00.10'
          ][::-1]
scale_perf = []
for scale in scales:
    test_paths = join('/media', 'root', 'easystore', 'experiments', 
                     'audiomnist', scale, 'AudioMNIST', 'data')
    full_file = glob.glob(join(test_paths, "*"))
    subj_perfs = []
    for filename in full_file:
        test_dat = np.load(filename)
        
        testX = test_dat[list(all_files.loc[(all_files.rec_split==9)&
                                            (all_files.subjid==int(filename.split("_")[-2])), 'ind_idx'])]
        testY = np.array(all_files.loc[(all_files.rec_split==9)&
                                       (all_files.subjid==int(filename.split("_")[-2])), 'digit'])
        dataset_test = torch.utils.data.TensorDataset(torch.Tensor(testX).cuda(), 
                                                       torch.LongTensor(testY).cuda())
        dataset_test = torch.utils.data.DataLoader(dataset_test, batch_size=2, shuffle=False)
        
        last_test_perf = test_model(model, 'cuda',
                                    dataset_test, batch_size=2)
        subj_perfs.append(last_test_perf)
    scoredict = {'perf':np.mean(subj_perfs),
                       'scal':float(scale)}
    print(scoredict)
    scale_perf.append(scoredict)
scale_perfs = pd.DataFrame(scale_perf)
scale_perfs.to_pickle(join("perf", "sithcon_hard_test_5032021.dill"))

{'perf': 0.9596666666666666, 'scal': 0.1}
{'perf': 0.9626666666666666, 'scal': 0.2}
{'perf': 0.959, 'scal': 0.4}
{'perf': 0.958, 'scal': 0.8}
{'perf': 0.9738983050847457, 'scal': 1.0}
{'perf': 0.9536666666666667, 'scal': 1.25}
{'perf': 0.9323333333333333, 'scal': 2.5}
{'perf': 0.6769999999999999, 'scal': 5.0}
{'perf': 0.3298305084745763, 'scal': 10.0}
